In [ ]:
#from llama_index.core.node_parser import SentenceSplitter

In [ ]:
!which python

In [ ]:
! pip install -U weaviate-client
! pip install pytube
! pip install youtube_transcript_api
! pip install llama_hub
! pip install llama_index

In [6]:
# running weaviate 
import os
import json
import weaviate

client = weaviate.Client("http://localhost:8080")


In [ ]:
#from llama_index import SimpleDirectoryReader

#derm_doc = SimpleDirectoryReader('../data/external').load_data()

In [ ]:
import whisper

model = whisper.load_model("base")

playlist_link = "https://www.youtube.com/playlist?list=PLRcGnj2vLUduM8elSDybdOVv4NO-nAz_9"


# Create a Playlist object
playlist = Playlist(playlist_link)

all_docs = []


for video_url in playlist.video_urls[:50]:
    try:
        # Transcribe using Whisper API
        transcript = transcribe(url=video_url, language="en", task="transcribe")
        
        all_docs.extend(transcript)
    except Exception as e:
        print(f"An error occurred for video {video_url}: {e}")

In [2]:
from pytube import Playlist
from llama_hub.youtube_transcript import YoutubeTranscriptReader
#from llama-index-readers-youtube-transcript import YoutubeTranscriptReader
from youtube_transcript_api import TranscriptsDisabled


loader = YoutubeTranscriptReader()
playlist_link = "https://www.youtube.com/playlist?list=PLRcGnj2vLUduM8elSDybdOVv4NO-nAz_9"


# Create a Playlist object
playlist = Playlist(playlist_link)

all_docs = []
for video_url in playlist.video_urls[:50]:
    try:
        # Load transcript for the current video
        documents = loader.load_data(ytlinks=[video_url])
        # Process the transcript as needed
        all_docs.extend(documents)
    except TranscriptsDisabled:
        print(f"Transcripts are disabled for the video: {video_url}")
    except Exception as e:
        print(f"An error occurred for video {video_url}: {e}")

Transcripts are disabled for the video: https://www.youtube.com/watch?v=YfZZRIjfp6E


In [7]:
output_folder = "../data/interim"
os.makedirs(output_folder, exist_ok=True)


#output file path
output_file_path = os.path.join(output_folder, "transcripts.json")

# list to store dictionaries for each document
json_data = []

# Iterate through all_docs and create a dictionary for each document
for doc in all_docs:
    doc_dict = {"Doc ID": doc.doc_id, "Text": doc.text}
    json_data.append(doc_dict)

# Save the list of dictionaries to a JSON file
with open(output_file_path, "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=2)

# Print a message indicating the successful save
print(f"Transcripts saved to: {output_file_path}")

Transcripts saved to: ../data/interim/transcripts.json


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
#from llama_index.core import Document, VectorStoreIndex

In [ ]:


#documentts = [Document(text=t) for t in all_docs]

In [ ]:
# creating nodes 
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(all_docs)

In [ ]:
import qdrant_client
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
)

In [ ]:

#from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import os 

In [ ]:
client = qdrant_client.QdrantClient(
    path="./qdrant_data"
)


vector_store = QdrantVectorStore(client=client, collection_name="tweets")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
## Weaviate

# construct vector store for embeddings
vector_store = WeaviateVectorStore(weaviate_client= client, index_name="Derm",  text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# set up the index
#index = VectorStoreIndex.from_documents(nodes, storage_context= storage_context)


In [ ]:
from llama_index.llms import Ollama
llm = Ollama(model="mistral")
service_context = ServiceContext.from_defaults(llm=llm,embed_model="local")


In [ ]:
# create the index; this will embed the documents and store them in the vector store
index = VectorStoreIndex.from_documents(all_docs,service_context=service_context,storage_context=storage_context)


In [ ]:
# query the index
query_engine = index.as_query_engine()
response = query_engine.query("What is this document about?. Give detail")
print(response)


In [ ]:
import weaviate
from llama_index.readers.weaviate.reader import WeaviateReader

reader = WeaviateReader("http://localhost:8080")

documents = reader.load_data(
     class_name="BlogPost", 
     properties=["content"], 
     separate_documents=True
)


In [ ]:

from llama_index import ListIndex


client = weaviate.Client(url="https://llamaindex-episode1-samtusdu.weaviate.network")

reader = WeaviateReader("https://llamaindex-episode1-samtusdu.weaviate.network")

query = """
{
  Get {
    BlogPost (
      bm25: {
        query: "What is ref2vec"
        properties: ["content"]
      },
      limit: 2
    ) {
      content
    }
  }
}
"""

documents = reader.load_data(graphql_query=query, separate_documents=True)

index = ListIndex.from_documents(documents)


query_engine = index.as_query_engine(response_mode="compact")
response = query_engine.query("what is ref2vec")
print(response)